### Import Libraries

In [ ]:
import re
import time
import pyperclip
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException

### Web Scraping Workflow
We use Python and Selenium to automate the collection of data from web pages

In [ ]:
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=chrome_options)
actions = ActionChains(driver)

list_link_sub = []
list_codes = []
list_desc = []
add1 = []
add2 = []
add3 = []
c_sub = 0
page = 0

# get first page
driver.get('https://web.reg.tu.ac.th/registrar/class_info.asp?lang=en')
web_link = "https://web.reg.tu.ac.th/registrar/"

# remember first page
 # seach click
time.sleep(25)
ele = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/table/tbody/tr[1]/td[2]/table/tbody/tr[7]/td[2]/table/tbody/tr/td/font[3]/input"))
)
ele.click()
first_tab = driver.current_window_handle
driver.maximize_window()
while True:
    try:
        extracted_texts = []
        time.sleep(1)

        # Find the link each subject
        rows = driver.find_elements(By.XPATH, "//tr[@valign='TOP']")
        for row in rows:
            c_sub += 1
            # print(c_sub)
            link_element = row.find_element(By.XPATH, ".//a[contains(@onclick, 'window.open')]")

            
            onclick_attribute = link_element.get_attribute("onclick")

            # Use regex to extract the URL from the 'onclick' attribute
            url_pattern = re.search(r"window\.open\('([^']+)'", onclick_attribute)

            extracted_url = url_pattern.group(1)  # Extract the URL
            pyperclip.copy(extracted_url)  # Copy the URL to the clipboard
            extracted_texts.append(extracted_url)  # Append the URL to the list

            # print(f"Extracted URL: {extracted_url}")
            list_link_sub.append(extracted_url)

        time.sleep(5)
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[1])

        # loop URL to open and extract data
        for i in extracted_texts:
            link = web_link+i
            driver.get(link)
            time.sleep(3)
            codes = driver.find_elements(By.XPATH, "/html/body/table[1]/tbody/tr/td[2]/table/tbody/tr[1]/td[1]/b/font")

            for code in codes:
                a = code.text
                list_codes.append(a)

            disc = driver.find_elements(By.XPATH, "/html/body/table[2]/tbody/tr/td[2]/font[2]")
            for i in disc:
                a = i.text
                list_desc.append(a)

            try:
                add = driver.find_elements(By.XPATH, "/html/body/table[1]/tbody/tr/td[2]/table/tbody/tr[6]")
                list = []
                for i in add:
                    a = i.text
                    list.append(a)
                add1.append(list)
            except:
                add1.append(None)

            try:
                add = driver.find_elements(By.XPATH, "/html/body/table[1]/tbody/tr/td[2]/table/tbody/tr[7]")
                listt = []
                for i in add:
                    a = i.text
                    listt.append(a)
                add2.append(listt)
            except:
                add2.append(None)
            
            try:
                add = driver.find_elements(By.XPATH, "/html/body/table[1]/tbody/tr/td[2]/table/tbody/tr[8]")
                listt = []
                for i in add:
                    print(i.text)
                    a = i.text
                    listt.append(a)
                add3.append(listt)
            except:
                add3.append(None)
                        

        # next page
        driver.switch_to.window(first_tab)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        try:
            if page == 0:
                driver.find_element(By.XPATH, '//td[2]/font/a').click()
                page += 1
            else:
                driver.find_element(By.XPATH, "//td[2]/font/a[2]").click()
                page += 1
                print(f"Moving to page {page}")
        except Exception as e:
            print(f"Could not find 'NEXT' button: {e}")
            break
            driver.quit()
    except:
        print("พัง")


### Data Processing
The scraped data is organized into a dictionary and converted into a Pandas DataFrame

In [67]:
# store data
dict = {"code":list_codes,"desc":list_desc,"add1":add1,"add2":add2,"add3":add3}
dataframe = pd.DataFrame(dict)
dataframe.head()

,code,desc,add1,add2,add3
0,AM101,Limit and continuity of single variable functi...,[disciplinary course(s): ไม่นับหน่วยกิตให้ผืที...,"[consecutive course(s): AM102, ST326]",[]
1,AM101,Limit and continuity of single variable functi...,[disciplinary course(s): ไม่นับหน่วยกิตให้ผืที...,"[consecutive course(s): AM102, ST326]",[]
2,AM101,Limit and continuity of single variable functi...,[disciplinary course(s): ไม่นับหน่วยกิตให้ผืที...,"[consecutive course(s): AM102, ST326]",[]
3,AM101,Limits and continuity of single variable funct...,[disciplinary course(s): หมายเหตุ : ไม่นับหน่ว...,"[consecutive course(s): AM102, AM201, ST326, S...",[]
4,AM101,Limits and continuity of single variable funct...,[disciplinary course(s): หมายเหตุ : ไม่นับหน่ว...,"[consecutive course(s): AM102, AM201, ST326, S...",[]


Change the datatype to ensure that it is string

In [71]:
dataframe[["add1","add2","add3"]] = dataframe[["add1","add2","add3"]].astype(str) 

Use API to create Database

In [2]:
# dsi314@dsi314.iam.gserviceaccount.com
import gspread
from google.oauth2.service_account import Credentials

We use Google Sheets as the first database

In [5]:
# api service verification
scopes = [
    "https://www.googleapis.com/auth/spreadsheets"
]
creds = Credentials.from_service_account_file("credentials.json",scopes=scopes)
client = gspread.authorize(creds)

#ระบุไฟล์ เปิดใช้งาน
sheet_id = "1ExtEUwVLi5Y_mmqxNd0oX7WrWzj8n6TeBLcQg7Xu0GU"
workbook = client.open_by_key(sheet_id) 

In [73]:
#using google sheet api
worksheet_list = map(lambda x: x.title, workbook.worksheets())
new_worksheet_name = "Sheet1"

if new_worksheet_name in worksheet_list:
    worksheet = workbook.worksheet(new_worksheet_name)
else:
    worksheet = workbook.add_worksheet(new_worksheet_name, rows=1000, cols=10)

data = [dataframe.columns.tolist()] + dataframe.values.tolist()
worksheet.update(data)

{'spreadsheetId': '1ExtEUwVLi5Y_mmqxNd0oX7WrWzj8n6TeBLcQg7Xu0GU',
 'updatedRange': 'Sheet1!A1:E774',
 'updatedRows': 774,
 'updatedColumns': 5,
 'updatedCells': 3870}